In [1]:
import arcpy

In [2]:
gdb = r"Q:\projects\Mwlrs\AEB_Prioritization_Tool\data\AEB_Analysis.gdb"
arcpy.env.workspace= gdb

In [4]:
#Add Major Stream Flag

fc = "streams_assessUnits"

# Create an update cursor to loop through and modify rows
with arcpy.da.UpdateCursor(fc, ["BLUE_LINE_KEY", "STREAM_ORDER", "MAJOR_STREAM_FLAG"]) as cursor:
    # Create a dictionary to track BLUE_LINE_KEY statuses
    blue_line_dict = {}

    # First pass to determine if any row for a BLUE_LINE_KEY has STREAM_ORDER >= 8
    for row in cursor:
        blue_line_key = row[0]
        stream_order = row[1]

        # Set status for BLUE_LINE_KEY in dictionary
        if stream_order >= 8:
            blue_line_dict[blue_line_key] = 'Y'
        elif blue_line_key not in blue_line_dict:
            blue_line_dict[blue_line_key] = 'N'

    # Reset the cursor for the second pass
    cursor.reset()

    # Second pass to update the MAJOR_STREAM_FLAG column
    for row in cursor:
        blue_line_key = row[0]

        # Update MAJOR_STREAM_FLAG based on the dictionary value
        row[2] = blue_line_dict[blue_line_key]
        cursor.updateRow(row)

print("Update complete!")

Update complete!


In [5]:
#Add Stream sensitivity

fc = "streams_assessUnits_ecosections"

print("Reading Sensitivity values...")
# Create a dictionary to store the sensitivity information by BLUE_LINE_KEY
sensitivity_dict = {}

# Read through the feature class to determine sensitivity
with arcpy.da.SearchCursor(fc, ["BLUE_LINE_KEY", "SUMMER_SNTVTY", "WINTER_SNTVTY"]) as cursor:
    for row in cursor:
        blue_line_key = row[0]
        summer_sensitivity = row[1]
        winter_sensitivity = row[2]
        
        if blue_line_key not in sensitivity_dict:
            sensitivity_dict[blue_line_key] = {"SUMMER": "Not Sensitive", "WINTER": "Not Sensitive"}
        
        if summer_sensitivity == "Sensitive":
            sensitivity_dict[blue_line_key]["SUMMER"] = "Sensitive"
        
        if winter_sensitivity == "Sensitive":
            sensitivity_dict[blue_line_key]["WINTER"] = "Sensitive"

print("Populating Stream Sensitivity values...")
# Update the feature class with the new sensitivity values
with arcpy.da.UpdateCursor(fc, ["BLUE_LINE_KEY", "STREAM_SUMMER_SNTVTY", "STREAM_WINTER_SNTVTY"]) as cursor:
    for row in cursor:
        blue_line_key = row[0]
        row[1] = sensitivity_dict[blue_line_key]["SUMMER"]
        row[2] = sensitivity_dict[blue_line_key]["WINTER"]
        cursor.updateRow(row)

print("Fields populated successfully!")

Reading Sensitivity values...
Populating Stream Sensitivity values...
Fields populated successfully!
